### Retriever

In [13]:
from langchain_community.vectorstores import FAISS as VectorStore
from langchain_community.embeddings import GPT4AllEmbeddings

embeddings = GPT4AllEmbeddings()
store = VectorStore.load_local("../../retrieve/vector_store", embeddings, allow_dangerous_deserialization=True)
retriever = store.as_retriever(search_kwargs={"k": 4})

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


### Model

In [14]:
import dotenv
dotenv.load_dotenv()

True

In [15]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id= "1bitLLM/bitnet_b1_58-3B"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
    llm = HuggingFacePipeline(pipeline=pipe)
except:
    llm = HuggingFacePipeline.from_model_id(
        model_id="databricks/dolly-v2-3b",
        task="text-generation",
        pipeline_kwargs={"max_new_tokens": 10},
    )

### Chains
https://python.langchain.com/docs/modules/memory/adding_memory_chain_multiple_inputs/

In [16]:
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

chain = load_qa_chain(llm=llm, chain_type="stuff", memory=memory, prompt=prompt)

In [17]:
query = "How install the chatbots"
docs = retriever.invoke(query)

chain.invoke({"input_documents": docs, "human_input": query}) #, return_only_outputs=True)

CPU times: user 5 µs, sys: 8 µs, total: 13 µs
Wall time: 26.9 µs


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


{'input_documents': [Document(page_content="The chatbots are in their respective folders with an application launcher `autorun.sh` to install each of them without specific knowledge.  \nOn Mac, open the terminal and type:\n```shell\ncd\n```\nDrag the **`folder`** containing the file `autorun.sh`, then press the Enter key (↩︎).  \n_If you have done it correctly, the **`~`** between your machine's name (`name@MacBook-Pro-of-Name`) and the **`%`** sign should display the name of the `folder` instead._  \nExecute the following line of code by pressing the Enter key (↩︎):\n```shell\nsh autorun.sh\n```\nWait a moment, the model should open in your default web browser.", metadata={'Header 1': 'Models', 'Header 2': 'Installation', 'Header 3': 'Streamlit & FastAPI'}),
  Document(page_content='The subfolders listed below contain key steps in our research for creating a functional, convenient, and maintainable chatbot.', metadata={'Header 1': 'Onboarding Bot Model', 'Header 2': 'Structure'}),
  D